In [1]:
import sys  # For running system level commands
import yaml  # For reading the config file
import os  # For path tools
import requests  # For symphony client
import symphony_client  # For connecting to Symphony region
from pip._vendor.distlib.compat import raw_input
import datetime
from pytz import timezone
import pytz  # To add timezone to datetime

In [2]:
!{sys.executable} -m pip install pyyaml

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
print(u"[INIT] Initialising script.")
# Configure environment
tz_utc = pytz.timezone("UTC") # Set timezone for data source
rootpath = os.path.dirname(os.path.realpath('__file__'))  # Get the root path

[INIT] Initialising script.


In [4]:
# Import config file data
with open(rootpath + '/config.yml', 'r') as stream:
    try:
        config = yaml.safe_load(stream)
        print(u" [\u2713] Config file loaded.")
    except yaml.YAMLError as exc:
        print(u" [\u2717] Could not load the config file.")
        print(exc)
        exit()
class textCol:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    PURPLE = '\033[95m'
    CYCAN = '\033[96m'
    END = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

 [✓] Config file loaded.


In [5]:
# Configure Stratoscale API connection
symp_url = "https://" + config["region_access"]["ipaddress"]
symp_domain = config["region_access"]["cloud_domain"]
symp_user = config["region_access"]["cloud_user"]
symp_password = config["region_access"]["cloud_password"]
symp_project = config["region_access"]["project"],
my_session = requests.Session()
my_session.verify = False

try:
    client = symphony_client.Client(url=symp_url, session=my_session)
    client_login = client.login(domain=symp_domain, username=symp_user, password=symp_password,project=symp_project)
    print(u" [\u2713] Stratoscale user region [{}] session established.".format(symp_url))
except:
    print(u" [\u2717] Could not connect to the Stratosacle region [{}] as user".format(symp_url))
    exit()

 [✓] Stratoscale user region [https://192.168.101.100] session established.


In [6]:
dbs = client.dbs.instance.list()
count = 0
for db in dbs:
    if db.role != 'standalone':
        count = count + 1
        if db.status.lower() == "active":
            status = "{}[Active]".format(textCol.GREEN)
        elif db.status.lower() == "pending":
            status = "{}[Pending]".format(textCol.PURPLE)
        elif db.status.lower() == "processing":
            status = "{}[Processing]".format(textCol.CYCAN)
        elif db.status.lower() == "stopped":
            status = "{}[Stopped]".format(textCol.BLUE)
        else:
            status = "{}Error".format(textCol.RED)
        db_details = "-------------------------------------------------------------------------------------------"
        db_details = ("{}\n [{}] {}{} [{}]{}\n  DB ID {}\t{}{}".format(
            db_details,
            count,
            textCol.BOLD,
            db.name,
            db.floating_ip,
            textCol.END,
            db.id,
            status,
            textCol.END
        ))
        db_data = client.dbs.instance.get(db.id)
        try:
            percent_used = 100 - int(db_data.stats.get("df/percent_bytes__free"))
            if percent_used > 85:
                percent_used = "{}{}[CRITICAL] - {}".format(textCol.BOLD, textCol.RED, percent_used)
            elif percent_used > 80:
                percent_used = "{}[ERROR] - {}".format(textCol.PURPLE, percent_used)
            elif percent_used > 70:
                percent_used = "{}[WARNING] - {}".format(textCol.YELLOW, percent_used)
            else:
                percent_used = "{}[OK] - {}".format(textCol.GREEN, percent_used)
        except:
            percent_used = "{}[No Data] - ?".format(textCol.YELLOW)
        print(db_details)
        print(db.role)

-------------------------------------------------------------------------------------------
 [1] Replica of ar_test_db01 [172.16.250.81]
  DB ID 31cd3ef2-c7e7-4f4f-bcb3-5caa9ddf800f	[Active]
replica
-------------------------------------------------------------------------------------------
 [2] dbs_postgresql11_01-replica [172.16.250.70]
  DB ID 6a7a4037-a401-4523-b558-3223ddb23980	[Active]
remote_replica
-------------------------------------------------------------------------------------------
 [3] dg-mysql-db [None]
  DB ID 8a73de13-816b-41b9-b2c6-7f8d6317351a	[Active]
master
-------------------------------------------------------------------------------------------
 [4] ar_test_db01 [172.16.250.114]
  DB ID 8c2c342c-5a4f-49d5-ad8d-924c54f35cb3	[Active]
master
-------------------------------------------------------------------------------------------
 [5] dg-mysql-db - Replica [None]
  DB ID cc8a466b-1279-48a2-abeb-0906f4d97f80	[Active]
replica
--------------------------------------

In [9]:
print "MYSQL"
db_data = client.dbs.instance.get("a5f089a9-e0f8-41aa-aba9-222e53005400")
print db_data

MYSQL
ResponseDict({u'log_external_endpoint_id': None, u'floating_ip_id': u'27be9618-b5ae-4946-80b2-8e9382da355c', u'superuser_enabled_at': None, u'updated_at': u'2019-10-20T08:46:02', u'master_user_name': u'admin', u'ldap_info': None, u'is_hidden': False, u'id': u'a5f089a9-e0f8-41aa-aba9-222e53005400', u'description': None, u'require_ssl': False, u'user_id': u'4be7a8c63f6a4980ad7fa01cdcf71462', u'log_output_description': None, u'replicas': None, u'allocated_storage': 512, u'affinity_type': u'soft_anti_affinity', u'security_group_id': u'500ac4f4-d0fb-4d2f-9c3d-9d34b4ba90d4', u'engine_revision_id': u'98837e03-5188-4e80-ad32-e33053bd08d7', u'tags_alt': [], u'role': u'standalone', u'master': None, u'project_id': u'30907850ff5a48a2add1532672944420', u'status': u'Active', u'account_id': u'8bee1925adc2438d9650d3525268f037', u'tags': [], u'external_network_id': None, u'remote_replication_port': None, u'floating_ip': u'172.16.250.40', u'superuser_enabled_by': None, u'remote_replication_host': 

In [7]:
print "MYSQL-REPLICA"
db_data = client.dbs.instance.get("d8ef60da-8042-4a73-84b2-59c1e51e6240")
print db_data

MYSQL
ResponseDict({u'log_external_endpoint_id': None, u'floating_ip_id': u'fb63cfb9-2b05-4d2f-808c-bb17be49aab4', u'superuser_enabled_at': None, u'updated_at': u'2019-10-21T15:07:02', u'master_user_name': u'admin', u'ldap_info': None, u'is_hidden': False, u'id': u'd8ef60da-8042-4a73-84b2-59c1e51e6240', u'description': None, u'require_ssl': False, u'user_id': u'4be7a8c63f6a4980ad7fa01cdcf71462', u'log_output_description': None, u'replicas': None, u'allocated_storage': 512, u'affinity_type': u'soft_anti_affinity', u'security_group_id': u'1e57b09a-4d3b-436d-b102-509d700e0093', u'engine_revision_id': u'98837e03-5188-4e80-ad32-e33053bd08d7', u'tags_alt': [], u'role': u'remote_replica', u'master': Munch({u'id': u'172.16.250.40:3306', u'name': u'Master on 172.16.250.40'}), u'project_id': u'30907850ff5a48a2add1532672944420', u'status': u'Active', u'account_id': u'8bee1925adc2438d9650d3525268f037', u'tags': [], u'external_network_id': u'fde62377-e981-4c45-b8c6-aa36a563ebb9', u'remote_replicati

In [10]:
print "POSTGRES"
db_data = client.dbs.instance.get("8d75b8e6-0a41-43b8-bb0a-787f81ed2537")
print db_data

POSTGRES
ResponseDict({u'log_external_endpoint_id': None, u'floating_ip_id': u'77ee4b1e-b458-4ad4-bc86-de73381ee801', u'superuser_enabled_at': None, u'updated_at': u'2019-10-20T08:46:01', u'master_user_name': u'admin', u'ldap_info': None, u'is_hidden': False, u'id': u'8d75b8e6-0a41-43b8-bb0a-787f81ed2537', u'description': None, u'require_ssl': False, u'user_id': u'4be7a8c63f6a4980ad7fa01cdcf71462', u'log_output_description': None, u'replicas': None, u'allocated_storage': 50, u'affinity_type': u'soft_anti_affinity', u'security_group_id': u'35c1ff5c-9d5c-4706-9ae6-1ad7cbabcda2', u'engine_revision_id': u'cda04041-0b95-4426-a1e9-a2c658c2d98e', u'tags_alt': [], u'role': u'standalone', u'master': None, u'project_id': u'30907850ff5a48a2add1532672944420', u'status': u'Active', u'account_id': u'8bee1925adc2438d9650d3525268f037', u'tags': [], u'external_network_id': None, u'remote_replication_port': None, u'floating_ip': u'172.16.250.47', u'superuser_enabled_by': None, u'remote_replication_host'

In [8]:
print "POSTGRES-REPLICA"
db_data = client.dbs.instance.get("6a7a4037-a401-4523-b558-3223ddb23980")
print db_data

POSTGRES
ResponseDict({u'log_external_endpoint_id': None, u'floating_ip_id': u'fb53928d-6a42-4246-af3a-ff4db4b56bdb', u'superuser_enabled_at': None, u'updated_at': u'2019-10-21T14:55:39', u'master_user_name': u'admin', u'ldap_info': None, u'is_hidden': False, u'id': u'6a7a4037-a401-4523-b558-3223ddb23980', u'description': None, u'require_ssl': False, u'user_id': u'4be7a8c63f6a4980ad7fa01cdcf71462', u'log_output_description': None, u'replicas': None, u'allocated_storage': 50, u'affinity_type': u'soft_anti_affinity', u'security_group_id': u'c5decbfa-5c40-49ff-b56b-0c6430afde5a', u'engine_revision_id': u'cda04041-0b95-4426-a1e9-a2c658c2d98e', u'tags_alt': [], u'role': u'remote_replica', u'master': Munch({u'id': u'172.16.250.47:5432', u'name': u'Master on 172.16.250.47'}), u'project_id': u'30907850ff5a48a2add1532672944420', u'status': u'Active', u'account_id': u'8bee1925adc2438d9650d3525268f037', u'tags': [], u'external_network_id': None, u'remote_replication_port': 5432, u'floating_ip': u